In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import matplotlib.pyplot as plt
from pydynmod.grid import Grid, ForceGrid
from pydynmod.snap import SnapShot, ParticleType

import mwtools.nemo
import pydynmod.analysesnap 
import math

%aimport -math,torch,numpy,matplotlib.pyplot,sys
potential=None

In [ ]:
fakesnap = SnapShot(positions=torch.zeros((1,3),dtype=torch.float32),
                    velocities=torch.zeros((1,3),dtype=torch.float32),
                    masses=torch.full((1,),0.3,dtype=torch.float32))

In [ ]:
potential=None
n=512
nz=512
potential=ForceGrid(n=(n,n,nz),
                    gridedges=torch.tensor([10.,10.,10.],dtype=torch.float32),
                    smoothing=0.1*20/n)
_=potential.griddata(fakesnap.positions,weights=fakesnap.masses,method='cic')
potential.grid_accelerations()

In [ ]:
positions=torch.zeros((1000,3),dtype=torch.float32)
positions[:,0]=torch.linspace(-10,10,1000)
positions[:,1]=0 
positions[:,2]=0
acc=potential.get_accelerations(positions)
plt.plot(positions[:,0].numpy(),-acc[:,0].numpy(),label='pydynmod')
plt.plot(potential.x.numpy(),-(potential.x*fakesnap.masses/potential.x.abs()**3).numpy())
plt.ylim(-2,2)

In [ ]:
snap=SnapShot('../inputmodels/M85_0',omega=1.)
particletype = torch.full((snap.n,),ParticleType.Star,dtype=torch.uint8)
particletype[snap.particletype==0]=ParticleType.DarkMatter
snap.particletype = particletype
omega,omegaerr = pydynmod.analysesnap.patternspeed(snap.stars)
snap.omega = torch.Tensor([omega]).type(torch.float32)
print(snap.omega)

In [ ]:
potential=None
n=512
nz=512
potential=ForceGrid(n=(n,n,nz),
                    gridedges=torch.tensor([10.,10.,10.],dtype=torch.float32),
                    smoothing=0.5*20/n)
_=potential.griddata(snap.positions,weights=snap.masses,method='cic')
potential.grid_accelerations()

In [ ]:
grav = mwtools.nemo.gravity_cartesian_grid(snap, potential.x,potential.y,[0.])

In [ ]:
positions=torch.zeros((len(potential.x),3),dtype=torch.float32)
positions[:,0]=grav['x']
positions[:,1]=grav['y'][255]
positions[:,2]=0
acc=potential.get_accelerations(positions)

In [ ]:
plt.plot(grav['x'].numpy(),grav['F'][:,255,0],label='gyrfalcON')
plt.plot(grav['x'].numpy(),-acc[:,0].numpy(),label='pydynmod')
plt.legend()
plt.xlim(-5,5)


In [ ]:
plt.plot(grav['x'].numpy(),grav['F'][:,255,0]+acc[:,0].numpy(),'.',label='pydynmod')
plt.legend()
plt.xlim(-2,2)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
f,axs = plt.subplots(1,3,sharex=True,sharey=True,figsize=(12,6))

ax=axs[0]
im=ax.contourf(grav['x'],grav['y'],grav['pot'])
ax.set_aspect('equal', 'box')
divider = make_axes_locatable(ax)
cax = divider.append_axes('top', size='5%', pad=0.05)
cb = f.colorbar(im, cax=cax, orientation='horizontal')
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')

ax=axs[1]
potslice = 0.5*(potential.pot[:,:,255]+potential.pot[:,:,255]).numpy()
im=ax.contourf(potential.x,potential.y,potslice)
ax.set_aspect('equal', 'box')
divider = make_axes_locatable(ax)
cax = divider.append_axes('top', size='5%', pad=0.05)
cb = f.colorbar(im, cax=cax, orientation='horizontal')
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')


ax=axs[2]
im=ax.contourf(potential.x,potential.y,
               potslice-grav['pot'] - np.mean(potslice-grav['pot']),
              cmap=plt.cm.get_cmap('coolwarm'))
ax.set_aspect('equal', 'box')
divider = make_axes_locatable(ax)
cax = divider.append_axes('top', size='5%', pad=0.05)
cb = f.colorbar(im, cax=cax, orientation='horizontal')
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')

f.tight_layout()